# Shape Files dos biomas

http://terrabrasilis.dpi.inpe.br/downloads/


# SEÇÃO TESTE 1

In [ ]:
##########################
# imports

import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import HeatMap, HeatMapWithTime, MarkerCluster
from shapely.geometry import Point, Polygon
import warnings
warnings.filterwarnings("ignore")

In [ ]:
##########################
# Funções e Procedures

# Carga de Shape Files
def shape2map(shp, mapa, bioma, color):
    # imports
    readedshape = gpd.read_file(shp)
    geoPath = readedshape.geometry.to_json()
    #poligonos = folium.features.GeoJson(geoPath)
    #mapa.add_child(poligonos)
    
    folium.GeoJson(geoPath,
               style_function=lambda x, color=color: {'color': color,
                                                      'weight': 0.5,
                                                      'fillOpacity': 0.25}
                      ).add_to(mapa)

    return mapa



In [ ]:
##########################
# criar o mapa base

my_map = folium.Map(
                location=(
                           -15.793836, -47.882721
                        ),
                zoom_start=4)
            

In [ ]:
# Driver
#gpd.io.file.fiona.drvsupport.supported_drivers['SHP'] = 'rw'


In [ ]:
##########################
# Polígono do Bioma Amazônico

# Arquivo SHP
border = './dados/amazonia_border/amazon_biome_border.shp'

# Adicionar ao Mapa
my_map = shape2map(border, my_map, 'Amazônia', 'green')

# Criar Data Frame
mapa_amazonia = gpd.read_file(border, driver='SHP')
mapa_amazonia["nome_bioma"] = 'Amazônia'


In [ ]:
##########################
# Polígono do Bioma Caatinga

# Arquivo SHP
border = './dados/caatinga_border/caatinga_border.shp'

# Adicionar ao Mapa
my_map = shape2map(border, my_map, 'Caatinga', 'yellow')

# Criar Data Frame
mapa_caatinga = gpd.read_file(border, driver='SHP')
mapa_caatinga["nome_bioma"] = 'Caatinga'

In [ ]:
##########################
# Polígono do Bioma Cerrado

# Arquivo SHP
border = './dados/cerrado_border/cerrado_border.shp'

# Adicionar ao Mapa
my_map = shape2map(border, my_map, 'Cerrado', 'Peru')

# Criar Data Frame
mapa_cerrado = gpd.read_file(border, driver='SHP')
mapa_cerrado["nome_bioma"] = 'Cerrado'


In [ ]:
##########################
# Polígono do Bioma Mata Atlântica

# Arquivo SHP
border = './dados/mata_atlantica_border/mata_atlantica_border.shp'

# Adicionar ao mapa
my_map = shape2map(border, my_map, 'Mata Atlântica', 'LawnGreen')

# Criar Data Frame
mapa_mata_atlantica = gpd.read_file(border, driver='SHP')
mapa_mata_atlantica["nome_bioma"] = 'Mata Atlântica'

In [ ]:
##########################
# Polígono do Bioma Pampa

# Arquivos SHP
border = './dados/pampa_border/biome_border.shp'

# Adicionar ao Mapa
my_map = shape2map(border, my_map, 'Pampa', 'DeepSkyBlue')

# Criar Data Frame
mapa_pampa = gpd.read_file(border, driver='SHP')
mapa_pampa["nome_bioma"] = "Pampa"

In [ ]:
##########################
# Polígono do Bioma Pantanal

# Arquivo SHP
border = './dados/pantanal_border/biome_border.shp'

# Adicionar ao Mapa
my_map = shape2map(border, my_map, 'Pantanal', 'MediumPurple')

# Criar Data Frame
mapa_pantanal = gpd.read_file(border, driver='SHP')
mapa_pantanal["nome_bioma"] = "Pantanal"

In [ ]:
##########################
# Tratar Data Frames dos Biomas

# Array com os Data Frames
arrays_df = [mapa_amazonia, mapa_cerrado, mapa_caatinga, mapa_mata_atlantica, mapa_pantanal, mapa_pampa]

# Concatenar Data Frames
df_mapa = pd.concat(arrays_df)

df_mapa.reset_index(inplace=True)
df_mapa.drop(columns=['index', 'id', 'bioma', 'cd_bioma', 'ID', 'NAME'], inplace=True)


In [ ]:
##########################
# Resultado da carga dos polígonos dos biomas

df_mapa

In [ ]:
##########################
# Carregar os dados de estações

# 

##########################
# Dados de teste
localizacoes = pd.read_csv('./dados/geo_localizacao.csv', sep=';')
localizacoes

In [ ]:
##########################
# Array com os pontos de localização latitude e longitude

# rever código para incluir informações das estações 

# dados de teste
pontos_geometricos = []
for xy in zip(localizacoes['longitude'], localizacoes['latitude']):
    pontos_geometricos.append(Point(xy))

In [ ]:
##########################
# formata dados de localização para processo de identificação dos pontos no polígono

# Formatação dos pontos
localizacoes_geo = gpd.GeoDataFrame(localizacoes,
                                    crs = {'init': 'epsg:4326'},
                                    geometry = pontos_geometricos)
# Impressão do resultado (cria o campo geometry)
localizacoes_geo

In [ ]:
##########################
# Validar pontos geográficos nos biomas

# iniciar variável
localizacoes_geo["pertence_ao_bioma"] = ""

# Varrer pontos e classificar biomas
for i in range(0, len(localizacoes_geo)):
    ponto = localizacoes_geo.geometry.loc[i]
    check = df_mapa.contains(ponto).values
    cont = 0
    bioma = ""
    for x in (check):
        if (x == True) and (cont == 0):
            bioma = 'Amazônia'
        elif (x == True) and (cont == 1):
            bioma = 'Cerrado'
        elif (x == True) and (cont == 2):
            bioma = 'Caatinga'
        elif (x == True) and (cont == 3):
            bioma = 'Mata Atlântica'
        elif (x == True) and (cont == 4):
            bioma = 'Pantanal'
        elif (x == True) and (cont == 5):
            bioma = 'Pampa'
        cont = cont + 1
    # Atualiza informação    
    localizacoes_geo["pertence_ao_bioma"].loc[i] = bioma

# imprimir resultado
localizacoes_geo

In [ ]:

pontos = localizacoes_geo[['latitude', 'longitude']].values.tolist()


marker_cluster = MarkerCluster().add_to(my_map)
for point in pontos:
    
    folium.Marker(
            location=[point[0], point[1]],
            popup= 'testes',
            icon=folium.Icon(color="green", icon="ok-sign"),
            ).add_to(marker_cluster)

my_map


# SEÇÃO TESTE 2

### SEGUIR ESTE EXEMPLO

O algoritimo de validação foi retirado deste exemplo

https://egmara-santos.medium.com/utilizando-python-para-verificar-coordenadas-geogr%C3%A1ficas-em-um-mapa-kml-c4d61541e5ce